# 02 - LangChain Agents

## 📚 學習目標

- ✅ 理解 Agent 的核心概念和運作原理
- ✅ 掌握 create_react_agent 的使用方法
- ✅ 學習如何定義和整合 Tools
- ✅ 實作 ReAct (Reasoning + Acting) 模式
- ✅ 建立國泰人壽智能客服 Agent

## ⏰ 預計時間: 50 分鐘

## 📦 環境設定

In [ ]:
# 安裝必要套件
!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install langchain-community==0.0.16
!pip install python-dotenv==1.0.0

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ 環境設定完成")
print(f"OpenAI API Key: {'已設定' if os.getenv('OPENAI_API_KEY') else '未設定'}")

## Part 1: Agent 核心概念

### 1.1 什麼是 Agent?

**Agent** 是能夠自主決策並使用工具完成任務的 AI 系統。

```
┌─────────────────────────────────────────┐
│         Agent 運作循環                   │
├─────────────────────────────────────────┤
│                                         │
│  1. 接收使用者輸入                       │
│         ↓                               │
│  2. 分析問題 (Reasoning)                │
│         ↓                               │
│  3. 選擇工具 (Tool Selection)           │
│         ↓                               │
│  4. 執行動作 (Action)                   │
│         ↓                               │
│  5. 觀察結果 (Observation)              │
│         ↓                               │
│  6. 重複 2-5 直到任務完成                │
│         ↓                               │
│  7. 返回最終答案                        │
│                                         │
└─────────────────────────────────────────┘
```

### Chain vs Agent

| 特性 | Chain | Agent |
|------|-------|-------|
| 流程 | 固定順序 | 動態決策 |
| 靈活性 | 低 | 高 |
| 工具使用 | 預設好的 | 自主選擇 |
| 適用場景 | 簡單流程 | 複雜任務 |
| 範例 | 翻譯流程 | 客服助理 |

### 1.2 ReAct 模式

**ReAct** = Reasoning (推理) + Acting (行動)

Agent 會在每一步:
1. **Thought**: 思考下一步要做什麼
2. **Action**: 選擇要使用的工具
3. **Action Input**: 工具的輸入參數
4. **Observation**: 觀察工具執行結果
5. 重複直到得出 **Final Answer**

In [ ]:
# ReAct 範例執行過程
example_process = """
Question: 查詢保單 CL202401001 的繳費狀況

Thought: 我需要先查詢這個保單的基本資訊
Action: policy_lookup
Action Input: CL202401001
Observation: 保單類型=終身壽險, 投保人=王小明, 年繳保費=50000

Thought: 現在我需要查詢繳費紀錄
Action: payment_history
Action Input: CL202401001
Observation: 2024-01 已繳, 2024-02 已繳, 2024-03 待繳

Thought: 我已經有足夠資訊回答問題了
Final Answer: 保單 CL202401001 目前繳費狀況良好,1-2月已正常繳費,3月份保費待繳。
"""

print(example_process)

## Part 2: 定義 Tools

### 2.1 什麼是 Tool?

Tool 是 Agent 可以使用的功能,例如:
- 查詢資料庫
- 呼叫 API
- 執行計算
- 搜尋資訊

### 2.2 使用 @tool 裝飾器定義工具

In [ ]:
from langchain.tools import tool
from typing import Dict

@tool
def policy_lookup(policy_id: str) -> str:
    """查詢保單基本資訊。
    
    Args:
        policy_id: 保單號碼
        
    Returns:
        包含保單類型、投保人、保費等資訊的字串
    """
    # 模擬資料庫查詢
    mock_db = {
        "CL202401001": {
            "type": "終身壽險",
            "owner": "王小明",
            "annual_premium": 50000,
            "status": "有效"
        },
        "CL202401002": {
            "type": "醫療險",
            "owner": "李小華",
            "annual_premium": 30000,
            "status": "有效"
        }
    }
    
    if policy_id in mock_db:
        info = mock_db[policy_id]
        return f"保單類型: {info['type']}, 投保人: {info['owner']}, 年繳保費: ${info['annual_premium']:,}, 狀態: {info['status']}"
    else:
        return f"找不到保單 {policy_id}"

@tool
def payment_history(policy_id: str) -> str:
    """查詢保單繳費紀錄。
    
    Args:
        policy_id: 保單號碼
        
    Returns:
        繳費紀錄字串
    """
    # 模擬繳費紀錄
    mock_payments = {
        "CL202401001": [
            {"period": "2024-01", "status": "已繳", "amount": 50000},
            {"period": "2024-02", "status": "已繳", "amount": 50000},
            {"period": "2024-03", "status": "待繳", "amount": 50000}
        ]
    }
    
    if policy_id in mock_payments:
        records = mock_payments[policy_id]
        result = []
        for r in records:
            result.append(f"{r['period']}: {r['status']} (${r['amount']:,})")
        return "\n".join(result)
    else:
        return f"找不到保單 {policy_id} 的繳費紀錄"

@tool
def premium_calculator(age: int, coverage: int) -> str:
    """計算保費。
    
    Args:
        age: 投保年齡
        coverage: 保額
        
    Returns:
        試算的年繳保費
    """
    # 簡單的保費計算公式 (實際會更複雜)
    base_rate = 0.02  # 基礎費率 2%
    age_factor = 1 + (age - 30) * 0.01  # 年齡係數
    annual_premium = int(coverage * base_rate * age_factor)
    
    return f"年齡 {age} 歲,保額 ${coverage:,} 的年繳保費約為 ${annual_premium:,}"

# 測試工具
print("🔧 測試工具功能:\n")
print("1. 保單查詢:")
print(policy_lookup.invoke({"policy_id": "CL202401001"}))
print("\n2. 繳費紀錄:")
print(payment_history.invoke({"policy_id": "CL202401001"}))
print("\n3. 保費試算:")
print(premium_calculator.invoke({"age": 35, "coverage": 2000000}))

### 2.3 使用 Tool 類別定義工具

In [ ]:
from langchain.tools import Tool

def claim_status_check(claim_id: str) -> str:
    """查詢理賠案件狀態"""
    mock_claims = {
        "CLM20240101": "審核中 - 預計 3 個工作天完成",
        "CLM20240102": "已核准 - 理賠金額 $50,000 已於 2024-03-15 匯款"
    }
    return mock_claims.get(claim_id, f"找不到理賠案件 {claim_id}")

# 使用 Tool 類別包裝
claim_tool = Tool(
    name="claim_status",
    func=claim_status_check,
    description="查詢理賠案件的處理狀態。輸入理賠編號,返回案件狀態。"
)

print("📋 理賠查詢:")
print(claim_tool.run("CLM20240102"))

## Part 3: 建立 ReAct Agent

### 3.1 使用 create_react_agent

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain import hub

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 準備工具列表
tools = [
    policy_lookup,
    payment_history,
    premium_calculator,
    claim_tool
]

# 載入 ReAct Prompt (LangChain Hub)
prompt = hub.pull("hwchase17/react")

# 建立 Agent
agent = create_react_agent(llm, tools, prompt)

# 建立 Agent Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # 顯示執行過程
    handle_parsing_errors=True,  # 處理解析錯誤
    max_iterations=5  # 最大迭代次數
)

print("✅ ReAct Agent 建立完成")

### 3.2 執行 Agent - 簡單查詢

In [ ]:
# 測試 1: 保單查詢
result = agent_executor.invoke({
    "input": "請查詢保單 CL202401001 的基本資訊"
})

print("\n" + "="*60)
print("🤖 Agent 回答:")
print("="*60)
print(result["output"])

### 3.3 執行 Agent - 多步驟推理

In [ ]:
# 測試 2: 需要多個工具的查詢
result = agent_executor.invoke({
    "input": "請幫我查詢保單 CL202401001 的繳費狀況,並告訴我下次該繳多少錢"
})

print("\n" + "="*60)
print("🤖 Agent 回答:")
print("="*60)
print(result["output"])

### 3.4 執行 Agent - 保費試算

In [ ]:
# 測試 3: 保費計算
result = agent_executor.invoke({
    "input": "我今年 40 歲,想要投保 300 萬的壽險,請幫我試算年繳保費"
})

print("\n" + "="*60)
print("🤖 Agent 回答:")
print("="*60)
print(result["output"])

## Part 4: 自訂 ReAct Prompt

### 4.1 客製化 Prompt 模板

In [ ]:
from langchain.prompts import PromptTemplate

# 建立國泰人壽專用的 ReAct Prompt
cathay_prompt = PromptTemplate.from_template("""
你是國泰人壽的專業客服 AI 助理,名叫「小泰」。

你的任務是協助客戶查詢保單、理賠、繳費等相關資訊。
請用專業、親切、有同理心的態度回答問題。

你可以使用以下工具:

{tools}

使用以下格式:

Question: 客戶的問題
Thought: 你應該思考要做什麼
Action: 要使用的工具,應該是 [{tool_names}] 其中之一
Action Input: 工具的輸入
Observation: 工具執行的結果
... (這個 Thought/Action/Action Input/Observation 可以重複 N 次)
Thought: 我現在知道最終答案了
Final Answer: 給客戶的最終回答

重要提醒:
1. 回答時請使用繁體中文
2. 保持專業但親切的口吻
3. 如果資訊不足,主動詢問客戶
4. 每次回答結束時,詢問客戶是否還有其他問題

開始!

Question: {input}
Thought: {agent_scratchpad}
""")

# 建立使用自訂 Prompt 的 Agent
custom_agent = create_react_agent(llm, tools, cathay_prompt)

custom_agent_executor = AgentExecutor(
    agent=custom_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("✅ 自訂 Agent 建立完成")

### 4.2 測試自訂 Agent

In [ ]:
result = custom_agent_executor.invoke({
    "input": "你好!我想了解一下我的保單 CL202401001 目前的狀況"
})

print("\n" + "="*60)
print("🤖 小泰的回答:")
print("="*60)
print(result["output"])

## Part 5: Agent 進階技巧

### 5.1 加入執行回調 (Callbacks)

In [ ]:
from langchain.callbacks import StdOutCallbackHandler

# 建立回調處理器
callback = StdOutCallbackHandler()

# 執行時傳入 callbacks
result = agent_executor.invoke(
    {"input": "查詢理賠案件 CLM20240102 的狀態"},
    {"callbacks": [callback]}
)

print("\n📊 執行結果:")
print(result["output"])

### 5.2 串流輸出 (Streaming)

In [ ]:
# 使用 stream 方法取得即時輸出
print("🔄 串流執行中...\n")

for chunk in agent_executor.stream(
    {"input": "45 歲投保 500 萬壽險的年繳保費是多少?"}
):
    if "output" in chunk:
        print(chunk["output"])
    elif "actions" in chunk:
        for action in chunk["actions"]:
            print(f"\n🔧 使用工具: {action.tool}")
            print(f"📥 輸入: {action.tool_input}")

### 5.3 錯誤處理

In [ ]:
# 測試錯誤處理
try:
    result = agent_executor.invoke({
        "input": "查詢不存在的保單 INVALID123"
    })
    print("\n🤖 Agent 回答:")
    print(result["output"])
except Exception as e:
    print(f"❌ 執行錯誤: {e}")

## Part 6: 實戰案例 - 完整的客服 Agent

### 6.1 擴充更多工具

In [ ]:
@tool
def product_recommendation(age: int, budget: int, needs: str) -> str:
    """根據客戶需求推薦保險商品。
    
    Args:
        age: 客戶年齡
        budget: 年度預算
        needs: 需求描述 (例如: 醫療保障、退休規劃、子女教育)
    """
    recommendations = {
        "醫療": "國泰醫療保險 - 提供住院、手術、門診等完整保障",
        "退休": "國泰利率變動型年金 - 穩健累積退休金",
        "教育": "國泰美滿人生 - 子女教育金專案",
        "壽險": "國泰終身壽險 - 終身保障,資產傳承"
    }
    
    result = []
    for key, product in recommendations.items():
        if key in needs:
            result.append(f"• {product}")
    
    if result:
        return f"根據您 {age} 歲,年度預算 ${budget:,},需求為『{needs}』,推薦:\n" + "\n".join(result)
    else:
        return f"根據您的需求『{needs}』,建議先與專業顧問詳談,了解更多選項。"

@tool  
def appointment_booking(name: str, phone: str, preferred_time: str) -> str:
    """預約與理財顧問諮詢。
    
    Args:
        name: 客戶姓名
        phone: 聯絡電話
        preferred_time: 希望時段
    """
    return f"已為 {name} 先生/小姐預約 {preferred_time} 的諮詢時段,我們會盡快致電 {phone} 確認。"

@tool
def faq_search(question: str) -> str:
    """搜尋常見問題。
    
    Args:
        question: 問題關鍵字
    """
    faqs = {
        "理賠": "理賠申請需準備: 1.理賠申請書 2.醫療單據 3.診斷證明 4.保單影本。一般 5-7 個工作天完成審核。",
        "解約": "保單解約需攜帶: 1.身分證 2.保單正本 3.存摺影本。可至各分公司辦理或線上申請。",
        "變更": "保單變更 (受益人、地址、電話) 可透過: 1.臨櫃辦理 2.客服專線 3.行動 APP 4.官網會員專區。"
    }
    
    for key, answer in faqs.items():
        if key in question:
            return answer
    
    return "找不到相關的常見問題,建議致電客服專線 0800-036-599 詢問。"

print("✅ 新增 3 個進階工具")

### 6.2 建立完整的客服 Agent

In [ ]:
# 完整工具列表
full_tools = [
    policy_lookup,
    payment_history,
    premium_calculator,
    claim_tool,
    product_recommendation,
    appointment_booking,
    faq_search
]

# 建立完整客服 Agent
customer_service_agent = create_react_agent(llm, full_tools, cathay_prompt)

cs_executor = AgentExecutor(
    agent=customer_service_agent,
    tools=full_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✅ 國泰人壽智能客服 Agent 準備就緒")
print(f"📋 可用工具數量: {len(full_tools)}")

### 6.3 測試完整客服場景

In [ ]:
# 場景 1: 產品諮詢
print("="*60)
print("📞 客戶諮詢 1: 產品推薦")
print("="*60)

result = cs_executor.invoke({
    "input": "我今年 35 歲,年收入約 100 萬,想要規劃醫療保障和退休金,預算一年 10 萬,有什麼建議嗎?"
})

print("\n🤖 小泰:")
print(result["output"])

In [ ]:
# 場景 2: 保單查詢 + FAQ
print("\n" + "="*60)
print("📞 客戶諮詢 2: 保單查詢與變更")
print("="*60)

result = cs_executor.invoke({
    "input": "我的保單是 CL202401001,想查一下繳費狀況,另外想問如何變更受益人?"
})

print("\n🤖 小泰:")
print(result["output"])

In [ ]:
# 場景 3: 理賠查詢 + 預約
print("\n" + "="*60)
print("📞 客戶諮詢 3: 理賠與預約")
print("="*60)

result = cs_executor.invoke({
    "input": "請幫我查理賠案件 CLM20240101 的進度,如果有問題我想預約顧問諮詢,我是陳大明,電話 0912-345-678,下週三下午方便"
})

print("\n🤖 小泰:")
print(result["output"])

## 🎯 練習題

### 練習 1: 建立保單健檢工具

**需求**:
1. 定義 `policy_health_check` 工具
2. 輸入保單號碼,輸出健檢報告
3. 檢查項目: 繳費狀況、保障是否足夠、是否有更好的方案
4. 整合到 Agent 中測試

In [ ]:
# TODO: 實作保單健檢工具

@tool
def policy_health_check(policy_id: str) -> str:
    """執行保單健檢。"""
    # TODO: 實作健檢邏輯
    pass

### 練習 2: 多語言客服 Agent

**需求**:
1. 修改 Prompt 讓 Agent 支援中英文
2. 根據客戶使用的語言自動切換
3. 測試中英文查詢

In [ ]:
# TODO: 建立多語言 Prompt
# TODO: 測試中英文互動

### 練習 3: 對話紀錄與分析

**需求**:
1. 建立工具記錄每次對話
2. 建立工具分析客戶意圖 (查詢/投訴/購買)
3. 建立工具產生客服報表

In [ ]:
# TODO: 實作對話紀錄系統

## ✅ 練習解答

### 練習 1 解答: 保單健檢工具

In [ ]:
@tool
def policy_health_check_solution(policy_id: str) -> str:
    """執行保單健檢,分析保障缺口和優化建議。
    
    Args:
        policy_id: 保單號碼
    """
    # 取得保單資訊 (實際會呼叫其他工具)
    policy_info = policy_lookup.invoke({"policy_id": policy_id})
    payment_info = payment_history.invoke({"policy_id": policy_id})
    
    # 模擬健檢分析
    report = f"""
📊 保單健檢報告 - {policy_id}
{'='*50}

✅ 保單狀態:
{policy_info}

💰 繳費狀況:
{payment_info}

📈 健檢結果:
1. 繳費狀況: 良好 ✅
2. 保障充足度: 中等 ⚠️
3. 費用合理性: 合理 ✅

💡 優化建議:
• 建議增加醫療險保障,現有保單偏重壽險
• 可考慮附加重大疾病險
• 保費水準合理,符合您的年齡區間

📞 如需詳細規劃,歡迎預約理財顧問諮詢。
"""
    
    return report

# 測試
result = policy_health_check_solution.invoke({"policy_id": "CL202401001"})
print(result)

## 📝 學習檢查清單

完成本 Notebook 後,你應該能夠:

- [ ] 理解 Agent 的核心概念和運作循環
- [ ] 區分 Chain 和 Agent 的差異
- [ ] 理解 ReAct 模式 (Reasoning + Acting)
- [ ] 使用 @tool 裝飾器定義工具
- [ ] 使用 Tool 類別封裝函數
- [ ] 使用 create_react_agent 建立 Agent
- [ ] 使用 AgentExecutor 執行 Agent
- [ ] 自訂 ReAct Prompt 模板
- [ ] 處理 Agent 執行錯誤
- [ ] 整合多個工具建立實用的客服系統

## 🎓 重點回顧

### Agent 核心組件
1. **LLM**: 負責推理和決策
2. **Tools**: Agent 可使用的功能
3. **Prompt**: 指導 Agent 行為的模板
4. **Agent Executor**: 執行 Agent 並管理循環

### ReAct 執行流程
```
Question → Thought → Action → Observation → ... → Final Answer
```

### 最佳實踐
1. 為每個工具撰寫清楚的 docstring
2. 設定合理的 max_iterations 避免無限循環
3. 啟用 handle_parsing_errors 處理解析錯誤
4. 使用 verbose=True 除錯時查看執行過程
5. 自訂 Prompt 以符合特定領域需求

### 下一步
繼續前往 `03-structured-output.ipynb` 學習結構化輸出 👉